In [1]:
from DataFetcher import DataFetcher
from USTs import USTs
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime
import plotly.graph_objects as go

In [2]:
auctions = DataFetcher().fetch_auction_data()
prices = DataFetcher().fetch_historical_UST_data(date=datetime.datetime(year=2025, month=6, day=26))

In [47]:
ust_set = USTs(auction_data=auctions,price_data=prices).get_current_UST_set(settlement_date=datetime.datetime(2025, 6, 27),
                                                                            get_ytms=True,
                                                                            include_FRNs=False,
                                                                            include_TIPS=False)

Merged auction and price data successfully
No missing or excess data
All CUSIPs are identical between DataFrames


In [46]:
filtered_set = ust_set[(ust_set['Days to expiry'] > 90) & (ust_set['Security type'] != 'Bill')]
years = ['2', '3', '5', '7', '10', '20', '30']
terms = [tenor + '-Year' for tenor in years]
fig = go.Figure()

for tenor in terms:
    subset = filtered_set[filtered_set['Original security term'] == tenor]
    fig.add_trace(go.Scatter(
        x=subset['Maturity date'],
        y=subset['EOD YTM']/100,
        mode='markers',
        name=tenor,
        marker=dict(
            size=6
        )
    ))

fig.update_layout(
    title='US Treasury Yield Curve (Days to Expiry > 90)',
    width=1100,
    height=600,
    margin=dict(l=20, r=20, t=50, b=20),
    xaxis_title='Maturity Date',
    yaxis_title='End-of-Day Yield to Maturity (YTM)',
    yaxis_tickformat='.2%',
    legend_title_text='Security Tenors',
    template='plotly_dark'
)

# Display the plot
fig.show()
